# 1. Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

2024-05-16 18:48:32.780159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 18:48:32.780262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 18:48:32.912020: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Define Image Size and Batch Size:

In [2]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 32  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [3]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 4. Get the Number of Classes from Folder Names:

In [4]:
folders = glob(train_dir + '/*')
print(len(folders))
num_classes = len(folders)

10


# 5. Load DenseNet121 Model:

In [8]:
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 6. Modify the output layer

In [9]:
x = densenet_model.output
x = GlobalAveragePooling2D()(x)

# Dense layers with dropout regularization
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(num_classes, activation='sigmoid')(x)

# 7. Freeze Convolutional Layers (Optional Fine-Tuning):

In [10]:
# Unfreeze the last few layers of the base model
for layer in densenet_model.layers[-10:]:
    layer.trainable = True

# 8. Create a new model with the new output layer

In [11]:
model = Model(inputs=densenet_model.input, outputs=predictions)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 11,893,578 (45.37 MB)

 Trainable params: 11,809,930 (45.05 MB)

 Non-trainable params: 83,648 (326.75 KB)

# 9. Compile the Model (Initial Training):

In [12]:
# compile the model
rmsprop_optimizer = RMSprop(learning_rate=0.0001, rho=0.9)
model.compile(optimizer=rmsprop_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Data Augmentation for Training and Validation:

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2 # 20% for validation
)
# for acutual val folder data
test_datagen = ImageDataGenerator(rescale=1./255)

# 11. Prepare Data Generators:

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 11. Early Stopping and Learning Rate Reduction:

In [16]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_resnet50_sigmoid_model_update1.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 12. Train the Model (Initial):

In [17]:
epoch = 20
history = model.fit(
    train_generator,
    epochs=epoch,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715885617.338924     101 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 794ms/step - accuracy: 0.4041 - loss: 1.6811
Epoch 1: val_loss improved from inf to 0.88742, saving model to tomato_resnet50_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 437s 926ms/step - accuracy: 0.4049 - loss: 1.6789 - val_accuracy: 0.8265 - val_loss: 0.8874 - learning_rate: 1.0000e-04
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - accuracy: 0.9163 - loss: 0.2632
Epoch 2: val_loss improved from 0.88742 to 0.11607, saving model to tomato_resnet50_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 142s 447ms/step - accuracy: 0.9163 - loss: 0.2631 - val_accuracy: 0.9745 - val_loss: 0.1161 - learning_rate: 1.0000e-04
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - accuracy: 0.9571 - loss: 0.1548
Epoch 3: val_loss improved from 0.11607 to 0.06259, saving model to tomato_resnet50_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 143s 447ms/step - accuracy: 0.9571 - loss: 0.1548 - val_accuracy: 0.9855 

# 13. Evaluate the Model after Initial Training:

In [18]:
score = model.evaluate(test_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 580ms/step - accuracy: 0.9796 - loss: 0.0877
Test loss (initial): 0.11718939989805222
Test accuracy (initial): 0.9769999980926514


# 14. Get the classification Report

In [19]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 367ms/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.93      1.00      0.96       100
                        Tomato___Early_blight       0.98      0.95      0.96       100
                         Tomato___Late_blight       0.98      0.96      0.97       100
                           Tomato___Leaf_Mold       0.99      1.00      1.00       100
                  Tomato___Septoria_leaf_spot       0.92      1.00      0.96       100
Tomato___Spider_mites Two-spotted_spider_mite       0.99      0.97      0.98       100
                         Tomato___Target_Spot       1.00      0.95      0.97       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       1.00      0.94      0.97       100
                 Tomato___Tomato_mosaic_virus       1.00      1.00      1.00       100
                             Tomato___healthy       1.00      1.00      1.00       100


# 15. Update the weights of the model

In [20]:
model.load_weights('/kaggle/working/tomato_resnet50_sigmoid_model_update1.weights.h5')